### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd 
from pathlib import Path as p
import win32com.client as win32

In [2]:
base_emails = pd.read_excel(r'caminho do arquivo excel Emails')
base_lojas = pd.read_csv(r'caminho do arquivo excel Lojas', encoding='latin1', sep=';')
base_vendas = pd.read_excel(r'caminho do arquivo excel Vendas')

### Passo 2 - juntar a base de lojas com a base de suas vendas, Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
base_vendas = pd.merge(base_vendas, base_lojas, on='ID Loja')

In [4]:
dicionario_lojas = {}

for loja in base_lojas['Loja']:
    dicionario_lojas[loja] = base_vendas.loc[base_vendas['Loja'] == loja, :]

In [7]:
dia_indicador = base_vendas['Data'].max()

### Passo 3 - Salvar a planilha na pasta de backup

In [9]:
caminho = p(r'Caminho da pasta de BackUp')
arquivos_lojas_backup = p.iterdir(caminho)
nomes_arquivos = [arquivo.name for arquivo in arquivos_lojas_backup]

for nome_loja in dicionario_lojas:
    if nome_loja not in nomes_arquivos:
        (caminho/nome_loja).mkdir()
    nome_arquivo = f'{nome_loja} {dia_indicador.day}-{dia_indicador.month}.xlsx'
    local_arquivo = caminho/nome_loja/nome_arquivo
    dicionario_lojas[nome_loja].to_excel(local_arquivo)

### Passo 4 - Calcular os indicadores e mandar email para o gerente

In [14]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtheprodutos_dia = 4
meta_qtheprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

for loja in dicionario_lojas:
    
    #pegamos apenas as colunas que vamos trabalhar, para diminiur o uso de memória
    dados_loja_ano = dicionario_lojas[loja][['Código Venda', 'Produto', 'Quantidade', 'Valor Unitário', 'Valor Final', 'Loja']]
    dados_loja_dia = dicionario_lojas[loja][['Código Venda', 'Produto', 'Quantidade', 'Valor Unitário', 'Valor Final', 'Loja']].loc[dicionario_lojas[loja]['Data'] == dia_indicador, :]

    faturamento_ano = dados_loja_ano['Valor Final'].sum()
    faturamento_dia = dados_loja_dia['Valor Final'].sum()

    qte_produtos_diferentes_ano = len(dados_loja_ano['Produto'].unique())
    qte_produtos_diferentes_dia = len(dados_loja_dia['Produto'].unique())


    valor_venda_por_cliente_dia = dados_loja_dia.groupby('Código Venda').sum()
    valor_venda_por_cliente_ano = dados_loja_ano.groupby('Código Venda').sum()
    ticketmedio_dia = valor_venda_por_cliente_dia['Valor Final'].mean()
    ticketmedio_ano = valor_venda_por_cliente_ano['Valor Final'].mean()

    nome_gerente = base_emails.loc[base_emails['Loja'] == loja, ['Gerente']].values[0][0]
    email_gerente = base_emails.loc[base_emails['Loja'] == loja, ['E-mail']].values[0][0]
   
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = email_gerente
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    
    if faturamento_dia >= meta_faturamento_dia:
        cor_faturamento_dia = '#008000'
    else:
        cor_faturamento_dia = '#FF0000'
    if faturamento_ano >= meta_faturamento_ano:
        cor_faturamento_ano = '#008000'
    else:
        cor_faturamento_ano = '#FF0000'
    if qte_produtos_diferentes_dia >= meta_qtheprodutos_dia:
        cor_qte_produtos_dia = '#008000' 
    else:
        cor_qte_produtos_dia = '#FF0000'       
    if qte_produtos_diferentes_ano >= meta_qtheprodutos_ano:
        cor_qte_produtos_ano = '#008000'      
    else:
        cor_qte_produtos_ano = '#FF0000'  
    if ticketmedio_dia >= meta_ticketmedio_dia:
        cor_ticketmedio_dia = '#008000'
    else:
        cor_ticketmedio_dia = '#FF0000'      
    if ticketmedio_ano >= meta_ticketmedio_ano:
        cor_ticketmedio_ano = '#008000'      
    else:
        cor_ticketmedio_ano = '#FF0000'

    mail.HTMLBody = f'''

    <p>Bom dia, {nome_gerente}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_faturamento_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qte_produtos_diferentes_dia}</td>
        <td style="text-align: center">{meta_qtheprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qte_produtos_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticketmedio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticketmedio_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_faturamento_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qte_produtos_diferentes_ano}</td>
        <td style="text-align: center">{meta_qtheprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qte_produtos_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticketmedio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_faturamento_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Alguém legal que fez isso</p>
    '''
    print(p.cwd())
    attachment  = p(f"'CAMINHO DA PASTA DE BACKUP'/{loja}/{loja} {dia_indicador.day}-{dia_indicador.month}.xlsx")
    mail.Attachments.Add(str(attachment))
    mail.Send()


d:\py\python_impressionador\PROJETO 1\Projeto AutomacaoIndicadores
d:\py\python_impressionador\PROJETO 1\Projeto AutomacaoIndicadores
d:\py\python_impressionador\PROJETO 1\Projeto AutomacaoIndicadores


### Passo 5 - Criar ranking para diretoria

In [11]:
faturamento_lojas = base_vendas.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

vendas_dia = base_vendas.loc[base_vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)


caminho = p(f"{p.cwd()}/Ranking Dia {dia_indicador.day}-{dia_indicador.month}.xlsx")
faturamento_lojas_dia.to_excel(f'{caminho}')

caminho = p(f"{p.cwd()}/Ranking Ano.xlsx")
faturamento_lojas_ano.to_excel(caminho)

#CRIA O FATURAMENTE DO DIA E DO ANO NO MESMO ARQUIVO EM QUE O PROGRAMA ESTÁ SENDO EXECUTADO

Passo 6 - Enviar e-mail para a diretoria

In [15]:
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = base_emails.loc[base_emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
'''

attachment  = p(f'{p.cwd()}/Ranking Ano.xlsx')
mail.Attachments.Add(str(attachment))
attachment  = p(f'{p.cwd()}/Ranking Dia {dia_indicador.day}-{dia_indicador.month}.xlsx')
mail.Attachments.Add(str(attachment))
mail.Send()

print('E-mail da Diretoria enviado')

E-mail da Diretoria enviado
